In [1]:
#!pip install transformers datasets rouge-score torch pandas


#I went from training two epochs on 170k steps to 500 steps aAnd still kaggle/colab crashes. 500 steps would get really bad results.

In [2]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch
from datasets import Dataset, DatasetDict
from rouge_score import rouge_scorer



In [3]:
with open("/content/train.csv", "r", encoding="utf-8") as f:
    for i, line in enumerate(f, start=1):
        try:
            pd.read_csv(pd.compat.StringIO(line))
        except Exception:
            print(f"Bad row at line {i}: {line}")
            break


Bad row at line 1: id,article,highlights



In [4]:
import pandas as pd

train_df = pd.read_csv("/content/train.csv", sep=",", quotechar='"', on_bad_lines="skip", engine="python")
val_df   = pd.read_csv("/content/validation.csv", sep=",", quotechar='"', on_bad_lines="skip", engine="python")
test_df  = pd.read_csv("/content/test.csv", sep=",", quotechar='"', on_bad_lines="skip", engine="python")

print(train_df.head())
print(train_df.columns)


                                         id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   
1  0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2  00027e965c8264c35cc1bc55556db388da82b07f   
3  0002c17436637c4fe1837c935c04de47adb18e9a   
4  0003ad6ef0c37534f80b55b4235108024b407f0b   

                                             article  \
0  By . Associated Press . PUBLISHED: . 14:11 EST...   
1  (CNN) -- Ralph Mata was an internal affairs li...   
2  A drunk driver who killed a young woman in a h...   
3  (CNN) -- With a breezy sweep of his pen Presid...   
4  Fleetwood are the only team still to have a 10...   

                                          highlights  
0  Bishop John Folda, of North Dakota, is taking ...  
1  Criminal complaint: Cop used his role to help ...  
2  Craig Eccleston-Todd, 27, had drunk at least t...  
3  Nina dos Santos says Europe must be ready to a...  
4  Fleetwood top of League One after 2-0 win at S...  
Index(['id', 'article', 'highlights'], dtype='obje

In [ ]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

In [ ]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)

max_input_length = 512
max_target_length = 128
def preprocess_function(batch):
    inputs = tokenizer(batch["article"], max_length=max_input_length, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["highlights"], max_length=max_target_length, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["article", "highlights"])


In [ ]:
model = BartForConditionalGeneration.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def compute_rouge(eval_pred):
    preds, labels = eval_pred
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    for pred, ref in zip(decoded_preds, decoded_labels):
        result = scorer.score(ref, pred)
        for key in scores:
            scores[key].append(result[key].fmeasure)

    # Average over dataset
    return {key: sum(vals)/len(vals) for key, vals in scores.items()}


In [3]:
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,   # adjust based on GPU
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,  # set higher for real training
    logging_dir="./logs",
    logging_steps=50,
    report_to=[]

)


NameError: name 'TrainingArguments' is not defined

In [2]:
import numpy as np

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_rouge,
)

NameError: name 'Trainer' is not defined

In [1]:
trainer.train()


NameError: name 'trainer' is not defined

In [ ]:
results = trainer.evaluate(tokenized_dataset["test"])
print("Test ROUGE Scores:", results)

In [ ]:
sample_text = test_df["article"].iloc[0]
inputs = tokenizer([sample_text], max_length=512, truncation=True, return_tensors="pt")
summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=150, min_length=50)
print("ARTICLE:", sample_text[:500], "...")
print("GENERATED SUMMARY:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))
print("REFERENCE SUMMARY:", test_df["highlights"].iloc[0])